In [1]:
%cd /content/drive/MyDrive/inference_SAM_video/sammed2d/SAMMed2D

/content/drive/MyDrive/inference_SAM_video/sammed2d/SAMMed2D


In [8]:
import sys
sys.path.append('/content/drive/MyDrive/inference_SAM_video/sammed2d/SAMMed2D/segment_anything')
from segment_anything.build_sam import sam_model_registry
from predictor_sammed import SammedPredictor
from argparse import Namespace
args = Namespace()
args.device = 'cuda'
args.image_size = 256
args.encoder_adapter = True
args.sam_checkpoint = "/content/drive/MyDrive/inference_SAM_video/ckpt/sam-med2d_b.pth"
sammed = sam_model_registry["vit_b"](args).to(args.device)
sammed.eval()
sammedpredictor = SammedPredictor(sammed)

True
*******load /content/drive/MyDrive/inference_SAM_video/ckpt/sam-med2d_b.pth


In [ ]:
import numpy as np
def compute_iou(pred_mask, gt_semantic_seg):
    in_mask = np.logical_and(gt_semantic_seg, pred_mask)
    out_mask = np.logical_or(gt_semantic_seg, pred_mask)
    iou = np.sum(in_mask) / np.sum(out_mask)
    return iou
def compute_dice(mask_gt, mask_pred):
    """Compute soerensen-dice coefficient.
    Returns:
    the dice coeffcient as float. If both masks are empty, the result is NaN
    """
    volume_sum = mask_gt.sum() + mask_pred.sum()
    if volume_sum == 0:
        return np.NaN
    volume_intersect = (mask_gt & mask_pred).sum()
    return 2*volume_intersect / volume_sum

In [ ]:
def repixel_value(arr, is_seg=False):
    if not is_seg:
        min_val = arr.min()
        max_val = arr.max()
        new_arr = (arr - min_val) / (max_val - min_val + 1e-10) * 255.
    return new_arr

In [ ]:
import os
import numpy as np
import cv2
import nibabel
import torch
resultpath = '/content/drive/MyDrive/inference_SAM_video/result/Brats/sammed2d_pointper103frame'
datapath = '/content/drive/MyDrive/lighting_sam_3d/data/Brats/resample/imagesTr'
promptpath = '/content/drive/MyDrive/inference_SAM_video/prompt/point/Brats'
visualize = '/content/drive/MyDrive/inference_SAM_video/visresult'
# classes = ['bg_change','blood','low_brightness','regular','smoke']
# direction = ['left','right']
for i in os.listdir(datapath):
  img3D = nibabel.load(os.path.join(datapath,i)).get_fdata()[np.newaxis,:,:,:]
  mask = nibabel.load(os.path.join(datapath,i)).get_fdata()[np.newaxis,:,:,:]
  img3d = torch.from_numpy(img3D)
  count = 0
  for j in range(img3D.shape[3]):
      if os.path.exists(os.path.join(resultpath,i,str(j),'dice.npy')):
        count += 1
        continue
      if 'prev_mask' not in locals():
        prev_mask = None
      positive_points = np.load(os.path.join(promptpath,i.split('.nii.gz')[0],str(count),'positive.npy'))
      negative_points = np.load(os.path.join(promptpath,i.split('.nii.gz')[0],str(count),'negative.npy'))
      input_points = np.concatenate([positive_points, negative_points])
      for m in range(len(input_points)):
        a = input_points[m][0]
        input_points[m][0] = input_points[m][1]
        input_points[m][1] = a
      input_labels = np.concatenate([np.ones(positive_points.shape[0]), np.zeros(negative_points.shape[0])])
      if len(positive_points) == 0:
        count += 1
        continue
      print(os.path.join(datapath,i,str(j)))
      input_image = torch.repeat_interleave(img3d, repeats=3, dim=0)
      img2d,gt2d = repixel_value(input_image[:,:,:,j]),repixel_value(mask[:,:,:,j])
      gt2d = gt2d.astype(bool)
      img2d = img2d.permute(1,2,0).numpy()
      img2d = img2d.astype(np.float16)
      sammedpredictor.set_image(img2d)
      if count % 103 == 0:
        positive_points = np.load(os.path.join(promptpath,i.split('.nii.gz')[0],str(count),'positive.npy'))
        negative_points = np.load(os.path.join(promptpath,i.split('.nii.gz')[0],str(count),'negative.npy'))
        input_points = np.concatenate([positive_points, negative_points])
        for m in range(len(input_points)):
          a = input_points[m][0]
          input_points[m][0] = input_points[m][1]
          input_points[m][1] = a
        input_labels = np.concatenate([np.ones(positive_points.shape[0]), np.zeros(negative_points.shape[0])])
        # input_box = np.load(os.path.join(promptpath,i,k,l.split('.')[0],'bbox_prompt.npy'))[0]
        masks, _, prev_mask = sammedpredictor.predict(
          point_coords=input_points,
          point_labels=input_labels,
          box = None,
          multimask_output=True,
        )
      else:
        masks, _, prev_mask = sammedpredictor.predict(
          point_coords=input_points,
          point_labels=input_labels,
          box = None,
          mask_input = prev_mask,
          multimask_output=True,
        )
      count += 1
      os.makedirs(os.path.join(resultpath,i,str(j)),exist_ok=True)
      dice = []
      for m in range(masks.shape[0]):
        np.save(os.path.join(resultpath,i,str(j),'mask' + str(m)+'.npy'),masks[m])
        m = masks[m].astype(bool)
        dice.append(compute_dice(gt2d,m))
        # iou.append(compute_iou(masks[m],gt))
      d = np.argsort(dice)
      prev_mask = prev_mask[[d[-1]]]
      np.save(os.path.join(resultpath,i,str(j),'dice.npy'),dice)
      # np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'iou.npy'),iou)

In [ ]:
import os
import cv2
resultpath = '/content/drive/MyDrive/inference_SAM_video/result/polypgen/sammed2d_pointperframe'
datapath = '/content/drive/MyDrive/inference_SAM_video/data/polypgen'
promptpath = '/content/drive/MyDrive/inference_SAM_video/prompt/point/polypgen'
visualize = '/content/drive/MyDrive/inference_SAM_video/visresult'
# classes = ['bg_change','blood','low_brightness','regular','smoke']
# direction = ['left','right']
for i in os.listdir(datapath):
  for j in range(len(os.listdir(os.path.join(datapath,i,'masks')))):
      count = 0
      # if os.path.exists(os.path.join(resultpath,i,j,k,l.split('.')[0],'dice.npy')):
      #   continue
      print(os.path.join(datapath,i,'masks',str(j)))
      img = cv2.imread(os.path.join(datapath,i,'images',str(j) + '.jpg'))
      p = np.load(os.path.join(promptpath,i,str(count),'positive.npy'))
      if len(p) == 0:
        continue
      sammedpredictor.set_image(img)
      if count % 1 == 0:
        positive_points = np.load(os.path.join(promptpath,i,str(count),'positive.npy'))
        negative_points = np.load(os.path.join(promptpath,i,str(count),'negative.npy'))
        input_points = np.concatenate([positive_points, negative_points])
        for m in range(len(input_points)):
          a = input_points[m][0]
          input_points[m][0] = input_points[m][1]
          input_points[m][1] = a
        input_labels = np.concatenate([np.ones(positive_points.shape[0]), np.zeros(negative_points.shape[0])])
        # input_box = np.load(os.path.join(promptpath,i,k,l.split('.')[0],'bbox_prompt.npy'))[0]
        masks, _, prev_mask = sammedpredictor.predict(
          point_coords=input_points,
          point_labels=input_labels,
          box = None,
          multimask_output=True,
        )
      else:
        masks, _, prev_mask = sammedpredictor.predict(
          point_coords=input_points,
          point_labels=input_labels,
          box = None,
          mask_input = prev_mask,
          multimask_output=True,
        )
      count += 1
      gt = np.load(os.path.join(datapath,i,'binarymask',str(j) + '.npy'))
      dice = []
      iou = []
      os.makedirs(os.path.join(resultpath,i,str(j)),exist_ok=True)
      for m in range(masks.shape[0]):
        np.save(os.path.join(resultpath,i,str(j),'mask' + str(m)+'.npy'),masks[m])
        mm = masks[m].astype(bool)
        dice.append(compute_dice(gt,mm))
        # iou.append(compute_iou(masks[m],gt))
      d = np.argsort(dice)
      prev_mask = prev_mask[[d[-1]]]
      np.save(os.path.join(resultpath,i,str(j),'dice.npy'),dice)
      # np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'iou.npy'),iou)

/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/0
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/1
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/2
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/3
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/4
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/5
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/6
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/7
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/8
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/9
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/10
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/11
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/12
/content/drive/MyDrive/inference_SAM_video/data/polypgen/seq14/masks/13
/c

In [ ]:
import os
import cv2
resultpath = '/content/drive/MyDrive/inference_SAM_video/result_sammed2d_bboxper10frame'
datapath = '/content/drive/MyDrive/inference_SAM_video/9'
promptpath = '/content/drive/MyDrive/inference_SAM_video/bbox_prompt'
visualize = '/content/drive/MyDrive/inference_SAM_video/visresult'
classes = ['bg_change','blood','low_brightness','regular','smoke']
direction = ['left','right']
for i in os.listdir(datapath):
  for j in classes:
    for k in direction:
      count = 0
      for c in range(len(os.listdir(os.path.join(datapath,i,j,k)))):
        # if os.path.exists(os.path.join(resultpath,i,j,k,l.split('.')[0],'dice.npy')):
        #   continue
        l = str(c) + '.png'
        print(os.path.join(datapath,i,j,k,l))
        img = cv2.imread(os.path.join(datapath,i,j,k,l))
        sammedpredictor.set_image(img)
        if count % 10 == 0:
          # positive_points = np.load(os.path.join(promptpath,i,k,str(count),'positive.npy'))
          # negative_points = np.load(os.path.join(promptpath,i,k,str(count),'negative.npy'))

          # input_points = np.concatenate([positive_points, negative_points])
          # for m in range(len(input_points)):
          #   a = input_points[m][0]
          #   input_points[m][0] = input_points[m][1]
          #   input_points[m][1] = a
          # input_labels = np.concatenate([np.ones(positive_points.shape[0]), np.zeros(negative_points.shape[0])])
          input_box = np.load(os.path.join(promptpath,i,k,l.split('.')[0],'bbox_prompt.npy'))[0]
          masks, _, prev_mask = sammedpredictor.predict(
            point_coords=None,
            point_labels=None,
            box = input_box,
            multimask_output=True,
          )
        else:
          masks, _, prev_mask = sammedpredictor.predict(
            point_coords=None,
            point_labels=None,
            box = input_box,
            mask_input = prev_mask,
            multimask_output=True,
          )
        count += 1
        gt = np.load(os.path.join(datapath,i,'ground_truth',k,l.split('.')[0] + '.npy'))
        dice = []
        iou = []
        os.makedirs(os.path.join(resultpath,i,j,k,l.split('.')[0]),exist_ok=True)
        for m in range(masks.shape[0]):
          np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'mask' + str(m)+'.npy'),masks[m])
          mm = masks[m].astype(bool)
          dice.append(compute_dice(gt,mm))
          # iou.append(compute_iou(masks[m],gt))
        d = np.argsort(dice)
        prev_mask = prev_mask[[d[-1]]]
        np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'dice.npy'),dice)
        # np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'iou.npy'),iou)

流式输出内容被截断，只能显示最后 5000 行内容。
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/100.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/101.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/102.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/103.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/104.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/105.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/106.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/107.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/108.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/109.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/110.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/111.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/112.png
/content/drive/MyDrive/inference_SAM_video/9/0/blood/right/113.png
/content/drive/MyDrive/inference_SA

In [ ]:
import os
import cv2
resultpath = '/content/drive/MyDrive/inference_SAM_video/result_sammed2d_prompt10perframe'
datapath = '/content/drive/MyDrive/inference_SAM_video/9'
promptpath = '/content/drive/MyDrive/inference_SAM_video/prompt_for_video_n'
visualize = '/content/drive/MyDrive/inference_SAM_video/visresult'
classes = ['bg_change','blood','low_brightness','regular','smoke']
direction = ['left','right']
for i in os.listdir(datapath):
  for j in classes:
    for k in direction:
      count = 0
      for count in range(len(os.listdir(os.path.join(datapath,i,j,k)))):
        l = str(count) + '.png'
        # if os.path.exists(os.path.join(resultpath,i,j,k,l.split('.')[0],'dice.npy')):
        #   continue

        print(os.path.join(datapath,i,j,k,l))
        img = cv2.imread(os.path.join(datapath,i,j,k,l))
        sammedpredictor.set_image(img)
        if count % 300 == 0:
          positive_points = np.load(os.path.join(promptpath,i,k,str(count),'positive.npy'))
          negative_points = np.load(os.path.join(promptpath,i,k,str(count),'negative.npy'))

          input_points = np.concatenate([positive_points, negative_points])
          for m in range(len(input_points)):
            a = input_points[m][0]
            input_points[m][0] = input_points[m][1]
            input_points[m][1] = a
          input_labels = np.concatenate([np.ones(positive_points.shape[0]), np.zeros(negative_points.shape[0])])

          masks, _, prev_mask = sammedpredictor.predict(
            point_coords=input_points,
            point_labels=input_labels,
            multimask_output=True,
          )
        else:
          masks, _, prev_mask = sammedpredictor.predict(
            point_coords=input_points,
            point_labels=input_labels,
            mask_input = prev_mask,
            multimask_output=True,
          )
        count += 1
        gt = np.load(os.path.join(datapath,i,'ground_truth',k,l.split('.')[0] + '.npy'))
        dice = []
        iou = []
        os.makedirs(os.path.join(resultpath,i,j,k,l.split('.')[0]),exist_ok=True)
        for m in range(masks.shape[0]):
          np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'mask' + str(m)+'.npy'),masks[m])
          mm = masks[m].astype(bool)
          dice.append(compute_dice(gt,mm))
          # iou.append(compute_iou(masks[m],gt))
        d = np.argsort(dice)
        prev_mask = prev_mask[[d[-1]]]
        np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'dice.npy'),dice)
        # np.save(os.path.join(resultpath,i,j,k,l.split('.')[0],'iou.npy'),iou)

/content/drive/MyDrive/inference_SAM_video/9/2/bg_change/left/0.png
/content/drive/MyDrive/inference_SAM_video/9/2/bg_change/left/1.png
/content/drive/MyDrive/inference_SAM_video/9/2/bg_change/left/2.png
/content/drive/MyDrive/inference_SAM_video/9/2/bg_change/left/3.png
/content/drive/MyDrive/inference_SAM_video/9/2/bg_change/left/4.png
/content/drive/MyDrive/inference_SAM_video/9/2/bg_change/left/5.png
/content/drive/MyDrive/inference_SAM_video/9/2/bg_change/left/6.png
/content/drive/MyDrive/inference_SAM_video/9/2/bg_change/left/7.png
/content/drive/MyDrive/inference_SAM_video/9/2/bg_change/left/8.png
/content/drive/MyDrive/inference_SAM_video/9/2/bg_change/left/9.png
/content/drive/MyDrive/inference_SAM_video/9/2/bg_change/left/10.png
/content/drive/MyDrive/inference_SAM_video/9/2/bg_change/left/11.png
/content/drive/MyDrive/inference_SAM_video/9/2/bg_change/left/12.png
/content/drive/MyDrive/inference_SAM_video/9/2/bg_change/left/13.png
/content/drive/MyDrive/inference_SAM_video/9

KeyboardInterrupt: 

In [ ]:
import os
resultpath = '/content/drive/MyDrive/inference_SAM_video/result_sammed2d_bboxper300frame'
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
classes = ['bg_change','blood','low_brightness','regular','smoke']
direction = ['left','right']
for j in classes:
  dice = []
  for k in direction:

    for i in os.listdir(resultpath):
      for l in os.listdir(os.path.join(resultpath,i,j,k)):
        if l == '.ipynb_checkpoints':
          continue
        # print(os.path.join(datapath,i,j,k,l))
        d = np.load(os.path.join(resultpath,i,j,k,l,'dice.npy'))
        dice.append(max(d))
        # IOU.append(max(np.load(os.path.join(resultpath,i,j,k,l,'iou.npy'))))
  print('average dice of ' + j + k + ' is ' + str(np.mean(dice)))
  # print('average iou of ' + j + k + ' is ' + str(np.mean(IOU)))
        # image = cv2.imread(os.path.join(datapath,i,j,k,l + '.png'))
        # mask = np.load(os.path.join(resultpath,i,j,k,l,'mask0.npy'))
        # print(image.shape)

        # gt = np.load(os.path.join(datapath,i,'ground_truth',k,l.split('.')[0] + '.npy'))
        # image[gt] = 13
        # image[mask] = 0
        # cv2_imshow(cv2.imread(os.path.join(datapath,i,j,k,l + '.png')))
        # point = np.load(os.path.join(promptp,i,k,l.split('.')[0],'positive.npy'))
        # for p in point:
        #   print(p)
        #   print(gt[p[0]][p[1]])
        #   cv2.circle(image, (int(p[1]), int(p[0])), 5, (0, 255, 0),thickness=-1)
        # cv2_imshow(image)
      # print('average dice of ' + i + j + k + ' is ' + str(np.mean(dice)))
      # print('average iou of ' + i + j + k + ' is ' + str(np.mean(IOU)))

average dice of bg_changeright is 0.3781695016204444
average dice of bloodright is 0.34524494155720276
average dice of low_brightnessright is 0.3365897191926342
average dice of regularright is 0.3672801237563488
average dice of smokeright is 0.34541073241933523


bboxprompt per frame sammed2d
average dice of bg_changeright is 0.5779024393451004
average dice of bloodright is 0.4322746148368252
average dice of low_brightnessright is 0.39949825443687736
average dice of regularright is 0.5556531491268908
average dice of smokeright is 0.5134616544899792

bbox 10 per frame sammed2d average dice of bg_changeright is 0.5122426258351951
average dice of bloodright is 0.3849499865076966
average dice of low_brightnessright is 0.37520207857219906
average dice of regularright is 0.4750860887350731
average dice of smokeright is 0.44706431427169907

bbox 300 per frame sammed2d
average dice of bg_changeright is 0.3781695016204444
average dice of bloodright is 0.34524494155720276
average dice of low_brightnessright is 0.3365897191926342
average dice of regularright is 0.3672801237563488
average dice of smokeright is 0.34541073241933523

In [ ]:
prompt 10 per frame
sammed2d
average dice of bg_changeright is 0.5737948496171511
average dice of bloodright is 0.44681407640154325
average dice of low_brightnessright is 0.36799369580500896
average dice of regularright is 0.5175356206138538
average dice of smokeright is 0.4839757434450899

In [ ]:
sam_jointmask_path = '/content/drive/MyDrive/inference_SAM_video/sammed2d_bboxper300frame'
resultpath = '/content/drive/MyDrive/inference_SAM_video/result_sammed2d_bboxper300frame'
classes = ['bg_change','blood','low_brightness','regular','smoke']
direction = ['left','right']
import shutil
import os
for j in classes:
  for i in os.listdir(resultpath):
    for k in direction:
      for l in range(len(os.listdir(os.path.join(resultpath,i,j,k)))):
        print(os.path.join(resultpath,i,j,k,str(l)))
        dice = np.load(os.path.join(resultpath,i,j,k,str(l),'dice.npy'))
        b = np.argsort(dice)
        os.makedirs(os.path.join(sam_jointmask_path,i,j,k,str(l)),exist_ok=True)
        shutil.move(os.path.join(resultpath,i,j,k,str(l),'mask' + str(b[-1]) + '.npy'),os.path.join(sam_jointmask_path,i,j,k,str(l),'mask.npy'))

流式输出内容被截断，只能显示最后 5000 行内容。
/content/drive/MyDrive/inference_SAM_video/result_sammed2d_bboxper300frame/2/low_brightness/right/100
/content/drive/MyDrive/inference_SAM_video/result_sammed2d_bboxper300frame/2/low_brightness/right/101
/content/drive/MyDrive/inference_SAM_video/result_sammed2d_bboxper300frame/2/low_brightness/right/102
/content/drive/MyDrive/inference_SAM_video/result_sammed2d_bboxper300frame/2/low_brightness/right/103
/content/drive/MyDrive/inference_SAM_video/result_sammed2d_bboxper300frame/2/low_brightness/right/104
/content/drive/MyDrive/inference_SAM_video/result_sammed2d_bboxper300frame/2/low_brightness/right/105
/content/drive/MyDrive/inference_SAM_video/result_sammed2d_bboxper300frame/2/low_brightness/right/106
/content/drive/MyDrive/inference_SAM_video/result_sammed2d_bboxper300frame/2/low_brightness/right/107
/content/drive/MyDrive/inference_SAM_video/result_sammed2d_bboxper300frame/2/low_brightness/right/108
/content/drive/MyDrive/inference_SAM_video/result_samme

In [ ]:
import os
resultpath = '/content/drive/MyDrive/inference_SAM_video/result_promptoncemedsam'
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
classes = ['bg_change','blood','low_brightness','regular','smoke']
direction = ['left','right']
for j in classes:
  dice = []
  for k in direction:

    for i in os.listdir(resultpath):
      for l in os.listdir(os.path.join(resultpath,i,j,k)):
        if l == '.ipynb_checkpoints':
          continue
        # print(os.path.join(datapath,i,j,k,l))
        d = np.load(os.path.join(resultpath,i,j,k,l,'dice.npy'))
        dice.append(max(d))
        # IOU.append(max(np.load(os.path.join(resultpath,i,j,k,l,'iou.npy'))))
  print('average dice of ' + j + k + ' is ' + str(np.mean(dice)))
  # print('average iou of ' + j + k + ' is ' + str(np.mean(IOU)))
        # image = cv2.imread(os.path.join(datapath,i,j,k,l + '.png'))
        # mask = np.load(os.path.join(resultpath,i,j,k,l,'mask0.npy'))
        # print(image.shape)

        # gt = np.load(os.path.join(datapath,i,'ground_truth',k,l.split('.')[0] + '.npy'))
        # image[gt] = 13
        # image[mask] = 0
        # cv2_imshow(cv2.imread(os.path.join(datapath,i,j,k,l + '.png')))
        # point = np.load(os.path.join(promptp,i,k,l.split('.')[0],'positive.npy'))
        # for p in point:
        #   print(p)
        #   print(gt[p[0]][p[1]])
        #   cv2.circle(image, (int(p[1]), int(p[0])), 5, (0, 255, 0),thickness=-1)
        # cv2_imshow(image)
      # print('average dice of ' + i + j + k + ' is ' + str(np.mean(dice)))
      # print('average iou of ' + i + j + k + ' is ' + str(np.mean(IOU)))

average dice of bg_changeright is 0.14882881989370794


KeyboardInterrupt: 